# Security Scraper tutorial

In [ ]:
import matplotlib.pyplot as plt

from yf_price import price_data
from utils import *
from plot_utils import *

##### Given a stock, a start date, an end date, and the reporting interval (cadence); we can fetch the security data from yahoo finance.

In [ ]:
stock = 'LMT'
start = str_to_UT('2021-07-15 00:00:00', timezone='US/Eastern')
end = str_to_UT('2021-07-16 00:00:00', timezone='US/Eastern')
interval = '5m'

yf_pd = price_data(stock, start, end, interval)

##### We can make a candle plot of the security with the built in plotting utilities, by passing the variable 'candle' to the plot() method.

In [ ]:
with plt.style.context('bmh'):
    fig, ax = plt.subplots(figsize=set_figsize(513))
    yf_pd.plot(fig, ax, 'candle')
    fig.tight_layout()

##### Fetch another ticker at the highest cadence with Pre and Post market data

In [ ]:
stock = 'PLTR'
start = str_to_UT('2021-07-15 00:00:00', timezone='US/Eastern')
end = str_to_UT('2021-07-16 00:00:00', timezone='US/Eastern')
interval = '1m'

yf_pd = price_data(stock, start, end, interval, PrePost=True)

##### Here we run all of the basic analysis that comes with the module

In [ ]:
# Calculate On Balance Volume
yf_pd.OBV()
# Calculate Moving Average
yf_pd.MA(60)
# Calculate Exponential Moving Average
yf_pd.EMA(120)
# Calculate Moving Average Convergence Divergence
yf_pd.MACD(short_span=30, long_span=360, signal_span=120)
# Calculate Return On Investment
yf_pd.ROI()
# Calculate Relative Change
yf_pd.RC()
# Calculate pivot, support, and resistance points
yf_pd.pivot_points(pivot_interval='1h')

##### From our basic analysis we can make a plot of a Moving Average with the support/resistance lines

In [ ]:
with plt.style.context('bmh'):
    fig, ax = plt.subplots(figsize=set_figsize(513))
    yf_pd.plot(fig, ax, 'close', npivots=3, label='Price', c=cc[0], lw=1)
    yf_pd.plot(fig, ax, 'MA', label='1h Moving Average', c=cc[1], lw=1)
    ax.legend()
    fig.tight_layout()

##### Here we also plot the MACD statistic, which is the standard MACD minus the signal. We've color code the background to indicate when MACD is suggesting to buy (green) vs sell (red)

In [ ]:
zero_crossings = list(
    np.where(np.diff(np.sign(yf_pd.data['MACD_sig'])))[0])+[-1]

with plt.style.context('bmh'):
    fig, ax = plt.subplots(2, 1, sharex=True, figsize=set_figsize(513),
                           gridspec_kw={'height_ratios': [3, 1]})
    yf_pd.plot(fig, ax[0], 'close', marker='.', label='Price',
               c=cc[0], ms=1, lw=1)
    yf_pd.plot(fig, ax[0], 'MA', marker='.', label='1h Moving Average',
               c=cc[1], ms=1, lw=1)
    yf_pd.plot(fig, ax[1], 'MACD_sig', marker='.', label='MACD-Signal',
               c=cc[0], ms=1, lw=1)
    ax[1].axhline(0, ls='--', c='r', lw=1, zorder=-1)
    ax[1].set_ylim([-1.03, 1.03])

    ax0_ylim, ax1_ylim = ax[0].get_ylim(), ax[1].get_ylim()
    colors = ['g', 'r']
    if np.sign(yf_pd.data['MACD_sig'].iloc[1]) < 0:
        colors, suggestions = colors[::-1], suggestions[::-1]
    for d in range(len(zero_crossings)-1):
        l, u = zero_crossings[d], zero_crossings[d+1]
        l, u = yf_pd.data.index[l], yf_pd.data.index[u]
        ax[0].fill_betweenx(ax0_ylim, l, u, alpha=0.1, color=colors[d%2],
                            zorder=-1)
        ax[1].fill_betweenx(ax1_ylim, l, u, alpha=0.1, color=colors[d%2],
                            zorder=-1)
    ax[0].legend(ncol=2)
    ax[0].set_ylabel('USD')
    ax[1].set_ylabel('MACD')
    ax[0].set_ylim(ax0_ylim)
    ax[1].set_ylim(ax1_ylim)
    ax[0].set_xlim([yf_pd.data.index[0], yf_pd.data.index[-1]])
    ax[1].set_xlim([yf_pd.data.index[0], yf_pd.data.index[-1]])
    ax[1].set_yticks([-1, 0, 1])
    fig.tight_layout()

##### Using MACD as the solo indicator is pretty lousy. Here we model buying `bvel` at every MACD buy interval, and selling `svel` at every MACD sell interval—closing out at end. This requires some finitely determine capital to achieve. We show the return on buying and holding with this capital, selling only at the end. 

In [ ]:
bvel, svel = 1, 1
capt, hold, capt_req = 0, 0, 0
for d, day in enumerate(yf_pd.data.index):
    price = yf_pd.data['close'].iloc[d]
    if yf_pd.data['MACD_sig'].iloc[d] > 0:
        hold += bvel
        capt -= bvel*price
    elif yf_pd.data['MACD_sig'].iloc[d] < 0 and hold >= 1:
        sell = (hold if hold <= svel else svel)
        hold -= sell
        capt += sell*price
    if capt < capt_req:
        capt_req = capt
print(f"Capital Required: ${-capt_req:.2f}")
print(f"Daily MACD Return: ${capt+hold*price:.2f}")
print(f"Buy and Hold Return: ${-capt_req*(yf_pd.data['ROI'].iloc[-1]):.2f}")

##### Here we show how to run analysis over multiple tickers at once

In [ ]:
stock = ['BA', 'AAPL', 'VTSAX']
start = str_to_UT('2021-01-01 00:00:00', timezone='US/Eastern')
end = str_to_UT('2021-07-16 00:00:00', timezone='US/Eastern')
interval = '1d'

yf_pd = price_data(stock, start, end, interval, PrePost=True)
# Calculate On Balance Volume
yf_pd.OBV()
# Calculate Moving Average
yf_pd.MA(5)
# Calculate Exponential Moving Average
yf_pd.EMA(9)
# Calculate Moving Average Convergence Divergence
yf_pd.MACD()
# Calculate Return On Investment
yf_pd.ROI()
# Calculate Relative Change
yf_pd.RC()
# Calculate pivot, support, and resistance points
yf_pd.pivot_points(pivot_interval='1wk')

##### Here we plot the ROI of each ticker, assuming they were all bought at the start date with no commission. If adjusted close data exist, then that is considered our initial buy in.

In [ ]:
with plt.style.context('bmh'):
    fig, ax = plt.subplots(figsize=set_figsize(513))
    yf_pd.plot(fig, ax, 'ROI', lw=1)
    ax.legend()
    fig.tight_layout()

##### We can also look at the on balance volume for all tickers. By default this is normalize so that multiple tickers can appear on the same scale; moreover, the quantitative value of OBV is not the statistic one typically considers.

In [ ]:
with plt.style.context('bmh'):
    fig, ax = plt.subplots(figsize=set_figsize(513))
    yf_pd.plot(fig, ax, 'OBV', lw=1)
    ax.legend()
    fig.tight_layout()

### Initalize fetcher

##### A tutorial of the fetching module itself, instead of being wrapped in our data analysis class

In [ ]:
from yf_fetcher import yf_fetcher

yf_fetcher = yf_fetcher()

## Fetch price history

The primary feature of this tool is to fetch the historical price history of stocks to perform analysis. This requires a stock ticker, a start and end date, and an invertval at which to sample the prices. To view the valid intervals recognized by YF's API one can call `yf_fetcher.valid_intervals()`.

In [ ]:
print(yf_fetcher.valid_intervals())

Now we are ready to fetch the historical price data for a few stocks. In this example we fetch Boeing (BA), and Vanguard's Total Stock Market Index Fund Admiral Shares (VTSAX) stock prices at a weekly interval. YF's API takes the dates in seconds since January 1st, 1970 (Unix Time). For those who don't measure time in such a manner they can use the string to Unix time function `str_to_UT()`, which takes the datetime as a string and converts to Unix time (note the stock market clock is set to the US/Eastern timezone).

Then we call the `yf_fetcher.fetch_price_history()` function as follows (note that it returns meta data, and price data which we save to the variables meta and data respectively).

In [ ]:
stock = ['BA', 'VTSAX']
start = str_to_UT('2009-12-28 00:00:00', timezone='US/Eastern')
end = str_to_UT('2010-02-08 00:00:00', timezone='US/Eastern')
interval = '1wk'

meta, data = yf_fetcher.fetch_price_history(stock, start, end, interval,
                                            div=False, split=False,
                                            PrePost=False)

Since we provided a list of stock tickers, data is a list of pandas dataframes. Let's look at the Boeing stock prices by printing `data['BA']`

In [ ]:
data['BA']

## Fetching the fundamentals

If you perfer to do analysis based off fundamentals, then the Yahoo Finanace website is probably a more convient interface. However, our fetcher can grab the same data if so desired. For a list of "modules" one can grab print the `yf_fetcher.valid_modules()` function.

In [ ]:
print(yf_fetcher.valid_modules())

As an example, let's grab all the data about Apple (APPL) with the `yf_fetcher.fetch_fundamentals()` function, which takes the stock ticker and the modules you wish to fetch (here we grab all by again calling the `valid_modules()` method).

In [ ]:
fundamentals = yf_fetcher.fetch_fundamentals('AAPL', yf_fetcher.valid_modules())

Now our fundamentals variables is a dictionary with all the modules we request, as an example let's look at the calendar events.

In [ ]:
fundamentals['calendarEvents']